In [ ]:
#default_exp unstructured2markup

In [ ]:
#no_test

#converts txt version of Genkins transcription into XML formatted according to SH specs, originally built for DSI entity extraction project

txt_path = input("Enter absolute path to txt version of trancription: ")
volume_title = input("Enter volume title: ")
volume_id = input("Enter volume identifier: ")
output_path = input("Enter file name for output: ")

volume = open(txt_path, 'r', encoding="utf-8")
output = open(output_path + ".xml", 'w', encoding="utf-8")

def writeHeader(title, out, id=None):
    out.write("<?xml version=\"1.0\" encoding=\"utf-8\"?>\n")
    out.write("<!--" + title + "-->\n")
    out.write("<ssda xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xmlns:xsd=\"http://www.w3.org/2001/XMLSchema\">\n")
    out.write("<volume>\n")
    out.write("<volumeTitle>" + title + "</volumeTitle>\n")
    if id != None:
        out.write("<volumeIdentifier>" + id + "</volumeIdentifier>")
    else:
        out.write("<volumeIdentifier />\n")
    out.write("<volumeGuid />\n")

def writeFooter(out):
    out.write("</text>\n")
    out.write("</item>\n")
    out.write("</volume>\n")
    out.write("</ssda>\n")

writeHeader(volume_title, output, volume_id)

in_entry = False
first_folio = True
folio_no = 1001

for line in volume:
    if ("Fol" in line) and (first_folio == True):
        first_folio = False
        output.write("<item>\n")
        output.write("<itemTitle>" + line[:line.find('\n')] + "</itemTitle>\n")
        output.write("<itemIdentifier>" + str(folio_no) + "</itemIdentifier>\n")
        folio_no += 1
        output.write("<itemGuid />\n")
        output.write("<text type=\"TranscribedText\">\n")
    elif line[:3] == "Fol" and (in_entry == False):
        output.write("</text>\n")
        output.write("</item>\n")
        output.write("<item>\n")
        output.write("<itemTitle>" + line[:line.find('\n')] + "</itemTitle>\n")
        output.write("<itemIdentifier>" + str(folio_no) + "</itemIdentifier>\n")
        folio_no += 1
        output.write("<itemGuid />\n")
        output.write("<text type=\"TranscribedText\">\n")
    elif line[:3] == "Fol" and in_entry:
        output.write("</entry>\n")
        output.write("</text>\n")
        output.write("</item>\n")
        output.write("<item>\n")
        output.write("<itemTitle>" + line[:line.find('\n')] + "</itemTitle>\n")
        output.write("<itemIdentifier>" + str(folio_no) + "</itemIdentifier>\n")
        folio_no += 1
        output.write("<itemGuid />\n")
        output.write("<text type=\"TranscribedText\">\n")
        output.write("<entry>\n")
    elif line[:3] == "[ma" and (in_entry == False):
        output.write("<entry>\n")
        output.write(line)
        in_entry = True
    elif in_entry and line[:3] == "[si":
        in_entry = False
        output.write(line)
        output.write("</entry>\n")
    elif line[:3] == "[ma":
        output.write("</entry>\n")
        output.write("<entry>\n")
        output.write(line)
    elif in_entry:
        line = line.replace('<', '')
        line = line.replace('>', '')
        output.write(line)

writeFooter(output)

volume.close()
output.close()

In [ ]:
#export

def universal_markup_converter(path_to_transcription, transcription_type):
    
    if transcription_type == "spatial_historian":
        inp = open(path_to_transcription,'r',encoding="utf-8")
        transcription = ''
        for line in inp:
            transcription += line
        typ = input("Enter record type: ")
        country = input("Enter country: ")
        state = input("Enter first-level administrative division: ")
        city = input("Enter city: ")
        institution = input("Enter institution: ")
        output_dir = input("Enter output directory: ")
        vol_id_loc = transcription.find("<volumeIdentifier>")
        volume_identifier = transcription[vol_id_loc + len("<volumeIdentifier>"):transcription.find('<',vol_id_loc + 1)]
        vol_titl_loc = transcription.find("<volumeTitle>")
        volume_title = transcription[vol_titl_loc + len("<volumeTitle>"):transcription.find('<',vol_titl_loc + 1)]
        img_type = input("Enter image file extensions: ")
        curr_image_id = int(input("Enter first transcribed image file name: "))
        inp.close()
        inp = open(path_to_transcription,'r',encoding="utf-8")
        
        output = open(output_dir,'w',encoding="utf-8")
        output.write("<?xml version=\"1.0\" encoding=\"utf-8\"?>\n")    
        output.write("<ssda xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xmlns:xsd=\"http://www.w3.org/2001/XMLSchema\">\n")
        output.write("<volume type=\"" + typ + "\" country=\"" + country + "\" state=\"" + state + "\" city=\"" + city + "\" institution=\"" + institution + "\" id=\"" + volume_identifier + "\" title=\"" + volume_title + "\">\n")
        
        in_entry = False
        for line in inp:
            if "<itemTitle>" in line:
                folio_start = line.find(' ')
                curr_image_number = line[folio_start + 1:line.find('.', folio_start)]                
                output.write("<image id=\"" + str(curr_image_id) + "\" type=\"" + img_type + "\" number=\"" + curr_image_number + "\">\n")
                curr_image_id += 1
                curr_entry = 1
            elif "<entry>" in line:
                in_entry = True
                output.write("<entry id=\"" + str(curr_entry) + "\">\n")
                curr_entry += 1            
            elif "</entry>" in line:
                in_entry = False
                output.write("</entry>\n")
            elif in_entry:
                output.write(line)
            elif "</item>" in line:
                output.write("</image>\n")            
        
        output.write("</volume>\n")
        output.write("</ssda>")
        
        inp.close()
        output.close()        
    else:
        print("that transcription type is not supported yet")
        
    return

In [ ]:
#no_test

universal_markup_converter("transcriptions/matanzas_baut_1.xml", "markup v1")

Enter record type: baptism
Enter country: Cuba
Enter first-level administrative division: Matanzas
Enter city: Matanzas
Enter institution: Catedral_de_San_Carlos_Borromeo
Enter output directory: transcriptions/15834.xml
Enter image file extensions: jpg
Enter first transcribed image file name: 1033


In [ ]:
#no_test

from nbdev.export import notebook2script
notebook2script()

Converted 12-ssda-xml-parser.ipynb.
Converted 31-collate-xml-entities-spans.ipynb.
Converted 33-split-data.ipynb.
Converted 41-generic-framework-for-spacy-training.ipynb.
Converted 42-initial-model.ipynb.
Converted 51-data-preprocessing.ipynb.
Converted 52-unstructured-to-markup.ipynb.
Converted 53-markup-to-spacy.ipynb.
Converted 61-prodigy-output-training-demo.ipynb.
Converted 62-full-model-application-demo.ipynb.
Converted 71-relationship-builder.ipynb.
